In [ ]:
!pip install kafka-python pyspark

In [ ]:
import findspark
findspark.init('/spark/spark-3.5.1-bin-hadoop3')
from pyspark import *
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, DateType, TimestampType, LongType
from pyspark.sql.types import ArrayType, DoubleType, BooleanType, DecimalType
from pyspark.sql.functions import regexp_extract, split, from_unixtime, col, avg, min, max
from pyspark.sql.functions import grouping

# Configuración del Streaming Context

En este caso hacemos un streaming context dentro del mismo `SparkContext` que teniamos. Solamente puede haber un `context` por JVM.

Configuramos el `StreamingContext` de tal forma que lea del socket cada 30 segundos

In [ ]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

sc = None
try:
    sc = SparkContext(appName="movielens")
except:
    sc = SparkContext.getOrCreate("movielens")

ssc = StreamingContext(sc, 30)  # Intervalo de 30 segundos

Lectura desde un Socket Text Stream

In [ ]:
lines = ssc.socketTextStream("localhost", 9999)

# Procesamiento de Datos en Tiempo Real

## Antes de correr el siguiente código

Para enviar datos al contexto de streaming, se puede utilizar el comando `netcat` en Unix (MacOS, Linux)

```
nc -lk 9999
```
Esto abre el puerto y permite que se le envíe información al programa.



In [ ]:
words = lines.flatMap(lambda line: line.split(" "))
wordCounts = words.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)
wordCounts.pprint()


Una vez que se ejecuta el codigo anterior, pasar a la ventana donde se ejecutó:

```
nc -lk 9999
```

Y escribir en el socket:

```
Este Este es un curso de Big Data de Big Data y Spark
```

Inicio del Contexto de Streaming

In [ ]:
ssc.start()
ssc.awaitTermination()


Una vez que salga el resultado anterior, agregarle:

```
Big Data
```

Referencia de la documentación: [Streaming Programming Guide](https://spark.apache.org/docs/latest/streaming-programming-guide.html)